In [1]:
import os
import sys
import json
from collections import namedtuple

import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm.notebook import trange, tqdm

from labels import labels


## Функции

In [15]:
def create_sityscapes_masks(n: int, needed_classes: list, classes_ids: dict, dataset_name: str,
                            labels_path: str, crop: tuple, result_shape: tuple) -> None:
    """
    Создание n-го количества масок необходимых классов
    """
    i = 0
    pbar = tqdm()
    pbar.reset(total=n)
    
    ladels_dirs = os.listdir(labels_path)[1:]
    # Проход по папкам train и val
    for ladels_dir in ladels_dirs:
        ladels_dir_path = os.path.join(labels_path, ladels_dir)
        labels_cities = os.listdir(ladels_dir_path)
        # Проход по папкам городов
        for labels_city in labels_cities:
            labels_city_path = os.path.join(ladels_dir_path, labels_city)
            images_names = os.listdir(labels_city_path)
            # Проход по изображениям масок
            for image_name in images_names:
                
                if 'labelIds' in image_name:
                    image_path = os.path.join(labels_city_path, image_name)
                    image = Image.open(image_path)
                    cropped_img = np.asarray(image.crop(crop))
                    
                    # Проход по классам
                    for needed_class in needed_classes:
                        mask = np.ma.masked_where(cropped_img == classes_ids[needed_class], cropped_img).mask.astype('uint8')
                        final_mask = np.zeros(result_shape).astype('uint8')
                        #Обработка пустой маски
                        if (mask == 1).sum() == 0:
                            Image.fromarray(final_mask).save(f'{dataset_name}/masks/{needed_class}/{image_name[:-20]}.png')
                        else:
                            for j in range(3):
                                final_mask[:,:,j] = mask
                            Image.fromarray(final_mask*255).save(f'{dataset_name}/masks/{needed_class}/{image_name[:-20]}.png') 
                    
                    pbar.update()
                    i += 1
                    if i == n:
                        sys.exit()
                    

In [16]:
def create_sityscapes_images(n: int, dataset_name: str, source_path: str, crop: tuple) -> None:
    """
    Создает n обрезанных изображений для датасета
    """
    i = 0
    pbar = tqdm()
    pbar.reset(total=n)
    
    ladels_dirs = os.listdir(source_path)[1:]
    # Проход по папкам train и val
    for ladels_dir in ladels_dirs:
        ladels_dir_path = os.path.join(source_path, ladels_dir)
        labels_cities = os.listdir(ladels_dir_path)
        # Проход по папкам городов
        for labels_city in labels_cities:
            labels_city_path = os.path.join(ladels_dir_path, labels_city)
            images_names = os.listdir(labels_city_path)
            # Проход по изображениям
            for image_name in images_names:
                image_path = os.path.join(labels_city_path, image_name)
                
                image = Image.open(image_path)
                cropped_img = image.crop(crop)
                cropped_img.save(f'{dataset_name}/images/{image_name[:-16]}.png')
                
                pbar.update()
                i += 1
                if i == n:
                    sys.exit()

## Создание датасета

In [4]:
classes_ids = {label.name:label.id for label in labels}

In [17]:
needed_classes = ['ground', 'road', 'sidewalk', 'building', 'fence',
                  'traffic light', 'traffic sign', 'vegetation', 'sky',
                  'car', 'truck', 'bus', 'train', 'motorcycle', 'bicycle']

labels_path = 'gtFine_trainvaltest/gtFine'
source_path = 'leftImg8bit_trainvaltest\leftImg8bit'
dataset_name = 'dataset_square1024_1000items'
n = 1000
crop = (512, 0, 1024+512, 1024)
shape = (1024, 1024, 3)


In [8]:
os.mkdir(dataset_name)
os.mkdir(f'{dataset_name}/images')
os.mkdir(f'{dataset_name}/masks')

In [9]:
for needed_class in needed_classes:
    os.mkdir(f'{dataset_name}/masks/{needed_class}')

In [ ]:
create_sityscapes_masks(n, needed_classes, classes_ids, dataset_name, labels_path, crop, shape)

In [14]:
create_sityscapes_images(n, dataset_name, source_path, crop)

aachen
bochum
bremen
cologne
darmstadt
dusseldorf


SystemExit: 

C:\Users\andre\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3426: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
